In [1]:
import pyranges as pr
import pandas as pd
import numpy as np


# approach

### Input last exons - selecting representative PAS/last exon:

le_ids can have multiple predictions, need to select a single one.
Strategy:
(starting with novel last exons used for quantification) 
1. Prefer events that match atlas
2. if multiple, select min atlas distance
3. If matches, s







Because quant regions 



# 

In [11]:
# input novel last exons used to generate combined reference of last exons
novel_le = pr.read_gtf("../data/papa/2023-03-29_papa_i3_cortical_upf1_zanovello_overlap_annotated.gtf")
# dexseq results df (used to extract cryptic events of each class)
dexseq_df = pd.read_csv("../data/papa/2023-05-24_i3_cortical_zanovello.all_datasets.dexseq_apa.results.processed.cleaned.tsv", sep="\t")
# refeence GTF used to identify novel last exons, quantify vs ref
ref_gtf = pr.read_gtf("../data/reference_filtered.gtf")
# 
tx2le = pd.read_csv("../data/papa/novel_ref_combined.tx2le.tsv", sep="\t")

# last exon quantification regions used as input to Salmon
quant_uniq_le = pr.read_gtf("../data/papa/novel_ref_combined.quant.last_exons.gtf")


info_df = pd.read_csv("../data/papa/novel_ref_combined.info.tsv", sep="\t")


In [8]:
print(quant_uniq_le.columns)
quant_uniq_le.subset(lambda df: ~df.transcript_id.str.startswith("ENST"))

Index(['Chromosome', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand',
       'Frame', 'gene_id', 'gene_name', 'transcript_id', 'exon_number',
       'region_rank', 'Start_ref', 'End_ref', 'transcript_id_ref',
       '3p_extension_length', 'event_type', 'ref_gene_id', 'ref_gene_name',
       'le_number', 'le_id'],
      dtype='object')


,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,...,region_rank,Start_ref,End_ref,transcript_id_ref,3p_extension_length,event_type,ref_gene_id,ref_gene_name,le_number,le_id
0,chr1,.,exon,24468985,24476667,.,+,.,PAPA.NT_19074713_S59.398,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
1,chr1,.,exon,24468985,24476667,.,+,.,PAPA.ctrl_ctrl_3.424,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
2,chr1,.,exon,24468985,24475256,.,+,.,PAPA.TDP43_19065409_S29.438,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
3,chr1,.,exon,24468985,24473773,.,+,.,PAPA.Cont-B_S2.334,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
4,chr1,.,exon,24468985,24474852,.,+,.,PAPA.DZ_curves_1_3.360,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36889,chrY,.,exon,13234576,13234826,.,-,.,PAPA.NT_19074729_S12.25635,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000183878.16,"UTY,UTY,UTY,UTY,UTY,UTY,UTY,UTY",7.0,ENSG00000183878.16_7
36890,chrY,.,exon,13234576,13234826,.,-,.,PAPA.TDP43-F_S6.20683,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000183878.16,"UTY,UTY,UTY,UTY,UTY,UTY,UTY,UTY",7.0,ENSG00000183878.16_7
36891,chrY,.,exon,13234576,13234826,.,-,.,PAPA.TDP43_19065413_S19.26168,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000183878.16,"UTY,UTY,UTY,UTY,UTY,UTY,UTY,UTY",7.0,ENSG00000183878.16_7
36892,chrY,.,exon,13234583,13234826,.,-,.,PAPA.DZ_curves_0_2.21889,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000183878.16,"UTY,UTY,UTY,UTY,UTY,UTY,UTY,UTY",7.0,ENSG00000183878.16_7


In [9]:
dexseq_df.columns

Index(['experiment_name', 'binID', 'groupID', 'featureID', 'exonBaseMean',
       'dispersion', 'stat', 'pvalue', 'padj', 'UsageCoefficient_base',
       'UsageCoefficient_contrast', 'log2fold_contrast_base', 'le_id',
       'gene.qvalue', 'contrast_name', 'gene_id', 'gene_name', 'event_type',
       'annot_status', 'transcript_id', 'chromosome', 'strand', 'start', 'end',
       'mean_PPAU_base', 'mean_PPAU_treatment', 'delta_PPAU_treatment_control',
       'simple_event_type'],
      dtype='object')

In [28]:
novel_le.columns

Index(['Chromosome', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand',
       'Frame', 'gene_id', 'transcript_id', 'event_type',
       '3p_extension_length', 'atlas_filter', 'nearest_atlas_distance', 'Name',
       'motif_filter', 'pas_motifs', 'condition_id', 'le_id', 'gene_name_ref',
       'gene_id_ref', 'mean_PPAU_base', 'mean_PPAU_treatment',
       'delta_PPAU_treatment_control', 'experiment_id', 'Cluster',
       'experiment_count', 'multiple_datasets', 'max_mean_PPAU'],
      dtype='object')

In [73]:
def select_rep_site(df):


    # First select sites of le_id that match atlas site (if applicable)
    if df["atlas_filter"].astype(int).sum() > 0:

        out_df = df[df["atlas_filter"].eq("1")]

        # 2. Within these, see if multiple different 3'ends predicted
        ### TODO: use Name (atlas ID) instead of 3' coordinate to match

        if out_df["Strand"].unique()[0] == "+":
            
            # plus strand - End coord = 3'end of last exon
            n_atlas = df["End"].nunique()

            if n_atlas > 1:
                # multiple different atlas sites predicted
                # see if somes sites were called in > 1 dataset - select those with max number of datasets called
                end3_c = out_df["End"].value_counts()
                end3_c = end3_c[end3_c.values == end3_c.max()]

                if len(end3_c) > 1:
                    # multiple 3'ends/ atlas sites predicted
                    raise Exception(f"{out_df['le_id_quant'].drop_duplicates()[0]} - le_id has multiple predicted atlas sites")
                                    
                else:
                    # 1 pas remaining
                    end3_coord = end3_c.index[0]

                    # Select a single row for le_id corresponding to selected PAS
                    out_df = out_df[out_df["End"] == end3_coord].sort_values(by="experiment_id").drop_duplicates()

                    decision_str = "atlas_max_datasets"


            else:
                # only single atlas site predicted - just select a single row
                end3_coord = out_df["End"].drop_duplicates().index[0]
                # Select a single row for le_id corresponding to selected PAS
                out_df = out_df[out_df["End"] == end3_coord].sort_values(by="experiment_id").drop_duplicates()

                decision_str = "atlas_1_pred"
                            
        else:
            # minus strand - Start coord = 3'end of last exon
            n_atlas = out_df["Start"].nunique()
            if n_atlas > 1:
                # multiple different atlas sites predicted
                # see if somes sites were called in > 1 dataset - select those with max number of datasets called
                end3_c = out_df["Start"].value_counts()
                end3_c = end3_c[end3_c.values == end3_c.max()]

                if len(end3_c) > 1:
                    # multiple 3'ends predicted
                    raise Exception(f"{out_df['le_id_quant'].drop_duplicates().iloc[0]} - le_id has multiple predicted atlas sites in same number of datasets")
                                    
                else:
                    # 1 pas remaining
                    end3_coord = end3_c.index[0]

                    # Select a single row for le_id corresponding to selected PAS
                    out_df = out_df[out_df["Start"] == end3_coord].sort_values(by="experiment_id").drop_duplicates()

                    decision_str = "atlas_max_datasets"

            else:
                # only single atlas site predicted - just select a single row
                end3_coord = out_df["Start"].drop_duplicates().index[0]
                # Select a single row for le_id corresponding to selected PAS
                out_df = out_df[out_df["Start"] == end3_coord].sort_values(by="experiment_id").drop_duplicates()

                decision_str = "atlas_1_pred"

        return out_df, decision_str    
    
    else:
        # TODO: implement motif filtering
        return df, "motif_only"
    
    

In [72]:
#### SELECTING REPRESENTATIVE PAS
# LEs have to be quantified
novel_le_df = novel_le.apply(lambda df: df.merge(tx2le, on="transcript_id", how="inner", suffixes=[None, "_quant"])).as_df()


i = 0

sel_dfs = []
sel_dict = {"atlas_1_pred": [],
            "atlas_max_datasets": [],
            "motif_only": []}

le_grpd = iter(novel_le_df.groupby("le_id_quant"))



while i < 100:
        le_id, df = next(le_grpd)
        print(le_id)
        out_df, decision_str = select_rep_site(df)
        sel_dfs.append(out_df)
        sel_dict[decision_str].append(le_id)
        i +=1  



                
    
print({dec: len(ids) for dec, ids in sel_dict.items()})


ENSG00000000003.15_2
ENSG00000000419.14_2
ENSG00000001036.14_3
ENSG00000001084.13_4
ENSG00000001167.15_2
ENSG00000001460.18_6
ENSG00000001461.17_6
ENSG00000001561.7_1
ENSG00000001629.10_3
ENSG00000001631.17_3


Exception: ENSG00000001631.17_3 - le_id has multiple predicted atlas sites in same number of datasets

In [86]:
print(novel_le_df[novel_le_df["le_id_quant"] == "ENSG00000001631.17_3"].columns)
novel_le_df[novel_le_df["le_id_quant"] == "ENSG00000001631.17_3"][["Start", "End", "atlas_filter", "Name", "le_id", "gene_name_ref"]]

Index(['Chromosome', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand',
       'Frame', 'gene_id', 'transcript_id', 'event_type',
       '3p_extension_length', 'atlas_filter', 'nearest_atlas_distance', 'Name',
       'motif_filter', 'pas_motifs', 'condition_id', 'le_id', 'gene_name_ref',
       'gene_id_ref', 'mean_PPAU_base', 'mean_PPAU_treatment',
       'delta_PPAU_treatment_control', 'experiment_id', 'Cluster',
       'experiment_count', 'multiple_datasets', 'max_mean_PPAU',
       'le_id_quant'],
      dtype='object')


,Start,End,atlas_filter,Name,le_id,gene_name_ref
14815,92198195,92200804,0,7:92197498:-,ENSG00000001631.17_3,KRIT1
14816,92198602,92200804,0,7:92199243:-,ENSG00000001631.17_3,KRIT1
14817,92199253,92200804,1,7:92199243:-,ENSG00000001631.17_3,KRIT1
14818,92199257,92200804,1,7:92199243:-,ENSG00000001631.17_3,KRIT1


In [ ]:
for le_id, df in novel_le_df.groupby("le_id_quant"):
        while i < 5:
                print(le_id)
                #1. select PAS passing atlas filter only (if applicable)
                print(df)
                print(df.Strand.unique()[0])
                print(len(df))

                if df["atlas_filter"].astype(int).sum() > 0:
                        out_df = df[df["atlas_filter"].eq("1")]

                        # 2. Within these, see if multiple different 3'ends predicted
                        if out_df["Strand"].unique()[0] == "+":
                                n_atlas = df["End"].nunique()

                                if n_atlas > 1:
                                        # multiple different atlas sites predicted
                                        # see if somes sites were called in > 1 dataset - select those with max number of datasets called
                                        end3_c = out_df["End"].value_counts()
                                        print("end3_c")
                                        print(end3_c)
                                        end3_c = end3_c[end3_c.values == end3_c.max()]
                                        print(end3_c)

                                else:
                                        # only single atlas site predicted - just select a single row
                                        pass
                            
                        else:
                                # minus strand - Start coord = 3'end of last exon
                                n_atlas = out_df["Start"].nunique()
                                if n_atlas > 1:
                                        # multiple different atlas sites predicted
                                        # see if somes sites were called in > 1 dataset - select those with max number of datasets called
                                        end3_c = out_df["Start"].value_counts()
                                        print("end3_c")
                                        print(end3_c)
                                        end3_c = end3_c[end3_c.values == end3_c.max()]
                                        print(end3_c)

                                        if len(end3_c) > 1:
                                            # multiple 3'ends predicted
                                            pass
                                    
                                        else:
                                            # 1 pas remaining
                                            end3_coord = end3_c.index[0]

                                            # Select a single row for le_id corresponding to selected PAS
                                            out_df = out_df[out_df["Start"] == end3_coord].sort_values(by="experiment_id").drop_duplicates()


                                else:
                                        # only single atlas site predicted - just select a single row
                                        end3_coord = out_df["Start"].drop_duplicates().index[0]
                                        # Select a single row for le_id corresponding to selected PAS
                                        out_df = out_df[out_df["Start"] == end3_coord].sort_values(by="experiment_id").drop_duplicates()
                
                print(len(out_df))
                
                i +=1